In [1]:
from torch_geometric.loader import DataLoader

from data import load_mnist_graph

from torch_geometric.nn import global_add_pool, global_mean_pool, global_max_pool

from layers import GraphSAGELayer
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd

from train import train_graph_classification, evaluate_graph_classification
from copy import deepcopy
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


%reload_ext autoreload
%autoreload 2

ImportError: /home/imenmahdi/.local/lib/python3.11/site-packages/torch/lib/../../nvidia/cusparse/lib/libcusparse.so.12: undefined symbol: __nvJitLinkComplete_12_4, version libnvJitLink.so.12

In [38]:
train_dataset, test_dataset = load_mnist_graph(subset=10000)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)



In [39]:
class GraphSAGE(nn.Module):
    def __init__(self, input_features, hidden_features, output_features, num_layers):
        super(GraphSAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(GraphSAGELayer(input_features, hidden_features, aggr='sum'))
        for _ in range(num_layers - 1):
            self.layers.append(GraphSAGELayer(hidden_features, hidden_features, aggr='sum'))
        self.linear = nn.Linear(hidden_features, output_features)
    def forward(self, x, edge_index, batch):
        for layer in self.layers:
            x = layer.propagate(x, edge_index)

        # readout
        x = global_mean_pool(x, batch)

        x = self.linear(x)
        return x
    

In [19]:

### Max number of epochs
max_epochs = 300

### Number of features
n_features, n_classes = 1, 10
hidden_size = 8

### DEFINE THE MODEL
basic_model = GraphSAGE(n_features, hidden_size, n_classes, num_layers=4).to(device)

### DEFINE LOSS FUNCTION
loss_fcn = nn.BCEWithLogitsLoss()

### DEFINE OPTIMIZER
optimizer = torch.optim.Adam(basic_model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=20, min_lr=0.0001)

### TRAIN THE MODEL
epoch_list, basic_model_scores = train_graph_classification(
    basic_model,
    loss_fcn,
    device,
    optimizer,
    max_epochs,
    train_loader,
    val_loader,
    n_classes,
)

100%|██████████| 300/300 [13:37<00:00,  2.72s/epoch, f1=0.763, loss=0.145]


In [41]:
train_dataset, test_dataset = load_mnist_graph(subset=6000)

# gather all data in one dataset (train/val/test)   
all_data = train_dataset + test_dataset
all_loader = DataLoader(all_data, batch_size=128, shuffle=False)

### Max number of epochs
max_epochs = 150

### Number of features
n_features, n_classes = 1, 10

### DEVICE GPU OR CPU : will select GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("\nDevice: ", device)


K = 12
best_score = 0
best_model = None
best_hyperparameters = None
results = pd.DataFrame(columns=['K', 'valid_score', 'test_score', 'lr', 'num_layers', 'hidden_size'])
for k in range(K):
    print(f"K-Fold: {k+1}/{K}")
    # split the data into train and test
    n = len(all_data)//K 
    test_range_index = np.arange(k*n, (k+1)*n)
    # all the rest for train
    train_range_index = np.setdiff1d(np.arange(len(all_data)), test_range_index)
    train_out = torch.utils.data.Subset(all_data, train_range_index)
    test = torch.utils.data.Subset(all_data, test_range_index)

    # randomly choose 10% of the train_out data to be the validation set
    n = len(train_out)//10
    val_range_index = np.random.choice(len(train_out), n, replace=False)
    val = torch.utils.data.Subset(train_out, val_range_index)
    train_in = torch.utils.data.Subset(train_out, np.setdiff1d(np.arange(len(train_out)), val_range_index))

    

    # create the dataloader
    train_in_loader = DataLoader(train_in, batch_size=128, shuffle=True)
    val_loader = DataLoader(val, batch_size=128, shuffle=False)
    test_loader = DataLoader(test, batch_size=128, shuffle=False)


    # define hyperparameters grid
    hyperparameters = {
        'lr': [0.01],
        'num_layers': [4],
        'hidden_size': [8, 16, 32]
    }

    best_score = 0
    for lr in hyperparameters['lr']:
        for num_layers in hyperparameters['num_layers']:
            for hidden_size in hyperparameters['hidden_size']:
                print(f"lr: {lr}, num_layers: {num_layers}, hidden_size: {hidden_size}")
                ### DEFINE THE MODEL
                basic_model = GraphSAGE(n_features, hidden_size, n_classes, num_layers).to(device)

                ### DEFINE LOSS FUNCTION
                loss_fcn = nn.BCEWithLogitsLoss()

                ### DEFINE OPTIMIZER
                optimizer = torch.optim.Adam(basic_model.parameters(), lr=lr)
                scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, min_lr=0.0001)

                ### TRAIN THE MODEL
                epoch_list, basic_model_scores = train_graph_classification(
                    basic_model,
                    loss_fcn,
                    device,
                    optimizer,
                    max_epochs,
                    train_in_loader,
                    val_loader,
                    n_classes,
                    save_best=True
                )

                # evaluate the model on the test set
                valid_score = evaluate_graph_classification(basic_model, device, val_loader)
                print(f"Valid F1-score: {valid_score}")

                # save the best model
                if valid_score > best_score:
                    best_score = valid_score
                    del best_model
                    best_model = deepcopy(basic_model)
                    best_hyperparameters = {
                        'lr': lr,
                        'num_layers': num_layers,
                        'hidden_size': hidden_size
                    }
                results.loc[len(results.index)] = [k, valid_score, 0, lr, num_layers, hidden_size]

                del basic_model
                del optimizer
                del scheduler
                torch.cuda.empty_cache()
                

    # evaluate the best model on the test set
    test_score = evaluate_graph_classification(best_model, device, test_loader)
    print(f"Test F1-score: {test_score}")

    # save the results in dataframe
    results.loc[len(results.index)] = [k, best_score, test_score, best_hyperparameters['lr'], best_hyperparameters['num_layers'], best_hyperparameters['hidden_size']]

# save the results in a csv file
results.to_csv('sage_mnist.csv', index=False)


Device:  cuda
K-Fold: 1/12
lr: 0.01, num_layers: 4, hidden_size: 8


100%|██████████| 150/150 [04:02<00:00,  1.62s/epoch, f1=0.697, loss=0.167]


Best model loaded
Valid F1-score: 0.7274762426900585
lr: 0.01, num_layers: 4, hidden_size: 16


100%|██████████| 150/150 [05:35<00:00,  2.24s/epoch, f1=0.739, loss=0.136]


Best model loaded
Valid F1-score: 0.7951845760233919
lr: 0.01, num_layers: 4, hidden_size: 32


...

Best model loaded
Valid F1-score: 0.6625548245614036
lr: 0.01, num_layers: 4, hidden_size: 16


100%|██████████| 150/150 [05:15<00:00,  2.10s/epoch, f1=0.76, loss=0.121] 


Best model loaded
Valid F1-score: 0.7830317982456141
lr: 0.01, num_layers: 4, hidden_size: 32


100%|██████████| 150/150 [08:02<00:00,  3.22s/epoch, f1=0.491, loss=0.242]


Best model loaded
Valid F1-score: 0.7511878654970761
Test F1-score: 0.8429987980769231


In [42]:
# save the results in a csv file
results.to_csv('sage_mnist.csv', index=False)

In [43]:
# see the mean and std of the results for the test set
test_results = results[results['test_score'] != 0].test_score
print(f"Mean test score: {test_results.mean()}")
print(f"Std test score: {test_results.std()}")

Mean test score: 0.7847618689903847
Std test score: 0.031246964921329355
